# Giriş İşlemleri

In [5]:
%%capture
!pip install kagglehub pandas datasets scikit-learn torch transformers[torch] matplotlib seaborn

In [6]:
import kagglehub
import pandas as pd
import os
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import Trainer, TrainingArguments
import numpy as np
import torch
import json

In [7]:
def is_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

In [8]:
# Kök dizin belirleme
if is_colab():
    from google.colab import drive
    drive.mount('/content/drive')
    kok_dizin = "/content/drive/MyDrive/spam_message_classifier_tr"
else:
    kok_dizin = os.getcwd()
print(f"Kök dizin: {kok_dizin}\n Not: eğer colab kullanıyorsanız, dizini değiştirmeniz gerekir.")

Mounted at /content/drive
Kök dizin: /content/drive/MyDrive/spam_message_classifier_tr
 Not: eğer colab kullanıyorsanız, dizini değiştirmeniz gerekir.


In [9]:
model_save_path = os.path.join(kok_dizin, "egitilen_model")
model_cikti_dizini = os.path.join(kok_dizin, "kilometre_tasi")
model_log_dizini = os.path.join(kok_dizin, "log")
eval_sonuc_json_path = os.path.join(model_log_dizini, "eval_sonuc.json")
train_sonuc_json_path = os.path.join(model_log_dizini, "train_sonuc.json")
if not os.path.exists(model_cikti_dizini):
    os.makedirs(model_cikti_dizini)
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

In [10]:
cihaz = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Cihaz: {cihaz}")

Cihaz: cuda


# Veri Kümesi Okuma

## Veri kümesi okuma fonksiyonu

In [11]:
def veri_kumesi_oku(veri_kumesi_adi: str) -> pd.DataFrame:
    """
    Kaggle veri kümesini indirir ve döndürür.

    Args:
        veri_kumesi_adi (str): Kaggle veri kümesinin yolu.
        gecici_yol (str): Geçici dosya yolu.
    Returns:
        pd.DataFrame: İndirilen veri kümesi.
    """
    # Kaggle veri kümesini indir
    gecici_yol = os.path.join(kagglehub.dataset_download(veri_kumesi_adi), "TurkishSMSCollection.csv")
    return pd.read_csv(gecici_yol, sep=";", encoding="utf-8")

## Veri kümesini oku

In [12]:
# Veri kümesi bilgileri
dataset_name = "onurkarasoy/turkish-sms-collection"

veri_kumesi = veri_kumesi_oku(dataset_name)
veri_kumesi.head(10)  # İlk 10 satırı göster

100%|██████████| 200k/200k [00:00<00:00, 66.2MB/s]

Extracting files...


,Message,Group,GroupText
0,125 lira,2,Normal
1,Baskanin aksam toplantısi fenaymis :),2,Normal
2,Bilal yalçnlara ne zaman gidiyoruz?,2,Normal
3,"BiP ile mesajlarimi aninda, daha eglenceli gon...",1,Spam
4,DIGITURKTEN FIRSAT! SiZE OZEL YIL SONUNA KADAR...,1,Spam
5,İyi ya dokuz eylül iyidir arş.gör falan kovala ;),2,Normal
6,İyiyim teşekkürler oturuyoruz nazarda arkadaşl...,2,Normal
7,Kapatamıyorun ayarlara girmem lazım :),2,Normal
8,Menüye girsem görünür mü acaba ??,2,Normal
9,Önemli değil hocam iyi akşamlar ;),2,Normal


## Veri kümesinin gereksiz kolonlarını silme

In [13]:
tohum_degeri = 571
# gereksiz sütunları kaldır
veri_kumesi = veri_kumesi[['Message', 'Group']]
# veri kümesini karıştır
veri_kumesi.head(10)  # İlk 10 satırı göster

,Message,Group
0,125 lira,2
1,Baskanin aksam toplantısi fenaymis :),2
2,Bilal yalçnlara ne zaman gidiyoruz?,2
3,"BiP ile mesajlarimi aninda, daha eglenceli gon...",1
4,DIGITURKTEN FIRSAT! SiZE OZEL YIL SONUNA KADAR...,1
5,İyi ya dokuz eylül iyidir arş.gör falan kovala ;),2
6,İyiyim teşekkürler oturuyoruz nazarda arkadaşl...,2
7,Kapatamıyorun ayarlara girmem lazım :),2
8,Menüye girsem görünür mü acaba ??,2
9,Önemli değil hocam iyi akşamlar ;),2


## Veri kümesini karıştırma

In [14]:
karismis_veri_kumesi = veri_kumesi.sample(frac=1, random_state=tohum_degeri).reset_index(drop=True)
karismis_veri_kumesi.head(10)  # İlk 10 satırı göster

,Message,Group
0,Aynen hem okul hem cafe,2
1,Alone'a gitmiyorsun sanırım öyle bir konuşma y...,2
2,Seni birazdan 7 kardeş grubuna ekleyeceğim fac...,2
3,Hayirli kandiller.,2
4,Nasıl böyle bi tuhaf oldu çocuk heyecanlandı,2
5,Is Bankasi Altin Bulusmalari 19-20 Mart'ta Yat...,1
6,Vatan'da bu haftasonu Samsung Galaxy S4 cep te...,1
7,http://bbk.im/1-SUBATA-OZEL-PRiMA-PREMiUM-CARE...,1
8,Dün burnuna bakıyor büyük diyor dedim benim bu...,2
9,Canım sonuçlar çıktı hiçbir anomali yok beyind...,2


## Veri kümesini eğitim ve sınama olarak parçalama

In [15]:
def veri_kumesini_oranli_bol(veri_kumesi, test_orani=0.1, dogrulama_orani=0.1, tohum_degeri=42):
    """
    Veri kümesini, Group kolonundaki değerlerin oranlarını koruyarak üçe böler.

    Args:
        veri_kumesi (pd.DataFrame): Bölünecek veri kümesi
        test_orani (float): Test veri kümesinin oranı (varsayılan: 0.1)
        dogrulama_orani (float): Doğrulama veri kümesinin oranı (varsayılan: 0.1)
        tohum_degeri (int): Random state değeri (tekrarlanabilirlik için)

    Returns:
        tuple: (egitim_veri_kumesi, dogrulama_veri_kumesi, test_veri_kumesi) - Oransal olarak bölünmüş veri kümeleri
    """
    # Benzersiz grup değerlerini bul
    gruplar = veri_kumesi['Group'].unique()

    egitim_liste = []
    dogrulama_liste = []
    test_liste = []

    # Her grup için ayrı ayrı bölme işlemi yap
    for grup in gruplar:
        # Sadece bu gruba ait verileri seç
        grup_verisi = veri_kumesi[veri_kumesi['Group'] == grup]

        # Bu grubu karıştır
        karisik_grup = grup_verisi.sample(frac=1, random_state=tohum_degeri)

        # Bölme noktalarını hesapla
        test_bolme_noktasi = int(len(karisik_grup) * test_orani)
        dogrulama_bolme_noktasi = int(len(karisik_grup) * dogrulama_orani)

        # Bu gruptan test, doğrulama ve eğitim verilerini ayır
        test_grubu = karisik_grup[:test_bolme_noktasi]
        dogrulama_grubu = karisik_grup[test_bolme_noktasi:test_bolme_noktasi+dogrulama_bolme_noktasi]
        egitim_grubu = karisik_grup[test_bolme_noktasi+dogrulama_bolme_noktasi:]

        # Listelere ekle
        test_liste.append(test_grubu)
        dogrulama_liste.append(dogrulama_grubu)
        egitim_liste.append(egitim_grubu)

    # Grupları birleştir
    sinama_veri_kumesi = pd.concat(test_liste).reset_index(drop=True)
    dogrulama_veri_kumesi = pd.concat(dogrulama_liste).reset_index(drop=True)
    egitim_veri_kumesi = pd.concat(egitim_liste).reset_index(drop=True)

    # Son bir karıştırma işlemi
    sinama_veri_kumesi = sinama_veri_kumesi.sample(frac=1, random_state=tohum_degeri).reset_index(drop=True)
    dogrulama_veri_kumesi = dogrulama_veri_kumesi.sample(frac=1, random_state=tohum_degeri).reset_index(drop=True)
    egitim_veri_kumesi = egitim_veri_kumesi.sample(frac=1, random_state=tohum_degeri).reset_index(drop=True)

    return egitim_veri_kumesi, dogrulama_veri_kumesi, sinama_veri_kumesi

In [16]:
egitim_veri, dogrulama_veri, sinama_veri = veri_kumesini_oranli_bol(karismis_veri_kumesi, tohum_degeri=tohum_degeri)
egitim_veri.head(5)  # İlk 5 satırı göster
dogrulama_veri.head(5)
sinama_veri.head(5)  # İlk 5 satırı göster

,Message,Group
0,Kampanya: Bircok Sampuan ve Yuz makyaj urunund...,1
1,"Günaydın Ali bey, ben yıllık izne ayrıldım ama...",2
2,Indirim: Watsons Buyuk Yilbasi Indirimi baslad...,1
3,Yok canim merak ettim sadece selam söyle annen...,2
4,TUGCULAR'da SEZON SONU INDIRIMI! Kabanlarda %7...,1


## Veri kümelerini eğitime hazır hale getirme fonksiyonu

In [17]:
# Veri kümesini hazırla
def verileri_hazirla(df, tokenizer):
    # Group değerlerini 0 ve 1'e dönüştür (Group=1 -> 0, Group=2 -> 1)
    # BERT sınıflandırma için 0-based indeksler kullanır
    labels = df["Group"].apply(lambda x: 0 if x == 1 else 1).tolist()

    # Dataset'e dönüştür
    dataset = Dataset.from_dict({
        "text": df["Message"].tolist(),
        "label": labels
    })

    # Tokenize işlemi
    def tokenize_function(examples):
        return tokenizer(
            examples["text"],
            padding="max_length",
            truncation=True,
            max_length=512,  # BERT için genellikle 512 kullanılır
        )

    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    return tokenized_dataset

## Sınama sonucu hesaplama fonksiyonu

In [18]:
# ölçüt hesaplama fonksiyonu
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Model Eğitim işlemleri

## Model yükleme

In [19]:
model_adi = "dbmdz/bert-base-turkish-128k-cased"
model = AutoModelForSequenceClassification.from_pretrained(model_adi)
tokenizer = AutoTokenizer.from_pretrained(model_adi)
model.to(cihaz)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/386 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/740M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-128k-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.18M [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(128000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [20]:
# Eğitim ve test veri kümelerini hazırla
egitim_veri_kumesi = verileri_hazirla(egitim_veri, tokenizer)
dogrulama_veri_kumesi = verileri_hazirla(dogrulama_veri, tokenizer)
sinama_veri_kumesi = verileri_hazirla(sinama_veri, tokenizer)

Map:   0%|          | 0/3803 [00:00<?, ? examples/s]

Map:   0%|          | 0/474 [00:00<?, ? examples/s]

Map:   0%|          | 0/474 [00:00<?, ? examples/s]

## Eğitim argümanlarını ve eğiticiyi ayarla

In [21]:
# Eğitim argümanlarını tanımla
training_args = TrainingArguments(
    output_dir=model_cikti_dizini,
    num_train_epochs=1,  # 1 epoch
    report_to=["tensorboard"],
    per_device_train_batch_size=8,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    warmup_steps=0,
    weight_decay=0.01,
    logging_dir=model_log_dizini,
    logging_steps=5,
    eval_strategy="steps",
    eval_steps=5,          # Her 5 adımda bir değerlendirme yapılacak
    save_strategy="steps",
    save_steps=5,          # Her 5 adımda bir model kaydedilecek
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    # Konsola çıktı için ek parametreler:
    logging_strategy="steps",
    logging_first_step=True,  # İlk adımın da loglanması için
)

# Eğiticiyi oluştur
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=egitim_veri_kumesi,
    eval_dataset=dogrulama_veri_kumesi,
    compute_metrics=compute_metrics,
)

## Eğitim ve sınama işlemleri

In [22]:
# Modeli eğit
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
5,0.425400,0.167835,0.978903,0.976852,1.000000,0.954751
10,0.108000,0.024620,1.000000,1.000000,1.000000,1.000000
15,0.021100,0.005929,1.000000,1.000000,1.000000,1.000000
20,0.008700,0.002975,1.000000,1.000000,1.000000,1.000000
25,0.005500,0.002224,1.000000,1.000000,1.000000,1.000000


TrainOutput(global_step=29, training_loss=0.1102212402198849, metrics={'train_runtime': 508.0162, 'train_samples_per_second': 7.486, 'train_steps_per_second': 0.057, 'total_flos': 976668237496320.0, 'train_loss': 0.1102212402198849, 'epoch': 0.9747899159663865})

In [23]:
train_results = trainer.state.log_history
with open(train_sonuc_json_path, "w") as f:
    json.dump(train_results, f, indent=4)

In [24]:
# Değerlendirme
eval_results = trainer.evaluate(sinama_veri_kumesi)
print(f"Değerlendirme sonuçları: {eval_results}")

with open(eval_sonuc_json_path, "w") as f:
    json.dump(eval_results, f, indent=4)
print(f"Değerlendirme sonuçları {eval_sonuc_json_path} dosyasına kaydedildi.")

Değerlendirme sonuçları: {'eval_loss': 0.023978080600500107, 'eval_accuracy': 1.0, 'eval_f1': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_runtime': 18.3277, 'eval_samples_per_second': 25.862, 'eval_steps_per_second': 25.862, 'epoch': 0.9747899159663865}
Değerlendirme sonuçları /content/drive/MyDrive/spam_message_classifier_tr/log/eval_sonuc.json dosyasına kaydedildi.


## Eğitilen modeli kaydetme

In [25]:
# Eğitilen modeli kaydet
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

('/content/drive/MyDrive/spam_message_classifier_tr/egitilen_model/tokenizer_config.json',
 '/content/drive/MyDrive/spam_message_classifier_tr/egitilen_model/special_tokens_map.json',
 '/content/drive/MyDrive/spam_message_classifier_tr/egitilen_model/vocab.txt',
 '/content/drive/MyDrive/spam_message_classifier_tr/egitilen_model/added_tokens.json',
 '/content/drive/MyDrive/spam_message_classifier_tr/egitilen_model/tokenizer.json')

# Görselleştirme

In [26]:
import matplotlib.pyplot as plt
import seaborn as sns

## Json okuma fonksiyonu

In [27]:
def json_dosya_oku(dosya_yolu):
    """JSON dosyasını okur ve içeriğini döndürür."""
    with open(dosya_yolu, 'r', encoding='utf-8') as f:
        return json.load(f)

## Çizim tarzını ayarla

In [28]:
plt.style.use('ggplot')
sns.set(font_scale=1.2)
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['font.family'] = 'DejaVu Sans'

## Görselleştirme fonksiyonları

In [29]:
def egitim_sureci_gorsellestirilmesi(train_results, cikti_dizini = model_log_dizini):
    """Eğitim sürecindeki ölçütleri görselleştirir."""
    # Eğitim ve değerlendirme adımlarını ayıkla
    train_steps = [x for x in train_results if 'loss' in x and 'eval_loss' not in x]
    eval_steps = [x for x in train_results if 'eval_loss' in x and 'epoch' in x]

    # Grafikler için gerekli verileri hazırla
    epochs_train = [step.get('epoch', 0) for step in train_steps]
    loss_train = [step.get('loss', 0) for step in train_steps]

    epochs_eval = [step.get('epoch', 0) for step in eval_steps]
    loss_eval = [step.get('eval_loss', 0) for step in eval_steps]
    accuracy_eval = [step.get('eval_accuracy', 0) for step in eval_steps]
    f1_eval = [step.get('eval_f1', 0) for step in eval_steps]
    precision_eval = [step.get('eval_precision', 0) for step in eval_steps]
    recall_eval = [step.get('eval_recall', 0) for step in eval_steps]

    # Kayıp (Loss) Grafiği
    plt.figure(figsize=(14, 8))
    plt.plot(epochs_train, loss_train, 'b-', marker='o', label='Eğitim Kaybı')
    plt.plot(epochs_eval, loss_eval, 'r-', marker='s', label='Doğrulama Kaybı')
    plt.xlabel('Döngü')
    plt.ylabel('Kayıp Değeri')
    plt.title('Eğitim ve Doğrulama Kaybının Döngü Üzerinden Değişimi', fontweight='bold')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(cikti_dizini, 'kayip_grafigi.png'), dpi=300, bbox_inches='tight')

    # Doğruluk, F1, Kesinlik ve Duyarlılık Grafiği
    plt.figure(figsize=(14, 8))
    plt.plot(epochs_eval, accuracy_eval, 'b-', marker='o', label='Doğruluk')
    plt.plot(epochs_eval, f1_eval, 'g-', marker='s', label='F1 Skoru')
    plt.plot(epochs_eval, precision_eval, 'r-', marker='^', label='Kesinlik')
    plt.plot(epochs_eval, recall_eval, 'm-', marker='D', label='Duyarlılık')
    plt.xlabel('Döngü')
    plt.ylabel('Ölçüt Değeri')
    plt.title('Doğrulama ölçütlerinin Döngü Üzerinden Değişimi', fontweight='bold')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(cikti_dizini, 'ölçütler_grafigi.png'), dpi=300, bbox_inches='tight')

    # Öğrenme Oranı Grafiği
    if 'learning_rate' in train_steps[0]:
        learning_rates = [step.get('learning_rate', 0) for step in train_steps]
        plt.figure(figsize=(14, 6))
        plt.plot(epochs_train, learning_rates, 'g-', marker='o')
        plt.xlabel('Döngü')
        plt.ylabel('Öğrenme Oranı')
        plt.title('Öğrenme Oranının Döngü Üzerinden Değişimi', fontweight='bold')
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(os.path.join(cikti_dizini, 'ogrenme_orani_grafigi.png'), dpi=300, bbox_inches='tight')
    plt.close('all')
def degerlendirme_sonuclari_gorsellestirilmesi(eval_sonuc, cikti_dizini = model_log_dizini):
    """Değerlendirme sonuçlarını görselleştirir."""
    # Değerlendirme olcutlerini çıkar
    olcutler = ['accuracy', 'f1', 'precision', 'recall']
    olcut_isimleri = ['Doğruluk', 'F1 Skoru', 'Kesinlik', 'Duyarlılık']

    olcut_degerleri = []
    for olcut in olcutler:
        key = f"eval_{olcut}"
        if key in eval_sonuc:
            olcut_degerleri.append(eval_sonuc[key])
        else:
            olcut_degerleri.append(0)  # Eksik olcut değeri için 0

    # olcut değerlerini göster
    plt.figure(figsize=(12, 8))
    bars = plt.bar(olcut_isimleri, olcut_degerleri, color=['#3274A1', '#E1812C', '#3A923A', '#C03D3E'])
    plt.ylim(0, 1.1)  # Y ekseni sınırları
    plt.title('Model Değerlendirme Sonuçları', fontweight='bold')
    plt.ylabel('Değer')

    # Bar üzerine değerleri ekleme
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.02,
                f'{height:.4f}', ha='center', va='bottom', fontsize=12)

    plt.grid(axis='y')
    plt.tight_layout()
    plt.savefig(os.path.join(cikti_dizini, 'degerlendirme_sonuclari.png'), dpi=300, bbox_inches='tight')

    # Kayıp ve çalışma süresi grafiği
    if 'eval_loss' in eval_sonuc and 'eval_runtime' in eval_sonuc:
        plt.figure(figsize=(10, 6))
        plt.bar(['Kayıp', 'Çalışma Süresi (sn)'],
                [eval_sonuc['eval_loss'], eval_sonuc['eval_runtime']],
                color=['#C03D3E', '#3274A1'])
        plt.title('Değerlendirme Kaybı ve Çalışma Süresi', fontweight='bold')
        plt.grid(axis='y')
        plt.tight_layout()
        plt.savefig(os.path.join(cikti_dizini, 'kayip_ve_sure.png'), dpi=300, bbox_inches='tight')
    plt.close('all')
def egitim_ozet_tablosu(train_results, eval_sonuc, cikti_dizini = model_log_dizini):
    """Eğitim sürecinin özet tablosunu oluşturur ve kaydeder."""
    # Son eğitim Ölçütleri
    son_egitim = None
    for step in reversed(train_results):
        if 'train_runtime' in step:
            son_egitim = step
            break

    if son_egitim:
        # Tablo verilerini hazırla
        data = {
            'Ölçüt': ['Eğitim Süresi (sn)', 'Ör./sn', 'Adım/sn', 'Toplam FLOPS',
                      'Son Kayıp', 'Son Döngü', 'Doğruluk', 'F1 Skoru'],
            'Değer': [
                son_egitim.get('train_runtime', 'N/A'),
                son_egitim.get('train_samples_per_second', 'N/A'),
                son_egitim.get('train_steps_per_second', 'N/A'),
                f"{son_egitim.get('total_flos', 0) / 1e12:.2f} T" if 'total_flos' in son_egitim else 'N/A',
                son_egitim.get('train_loss', 'N/A'),
                son_egitim.get('epoch', 'N/A'),
                eval_sonuc.get('eval_accuracy', 'N/A'),
                eval_sonuc.get('eval_f1', 'N/A')
            ]
        }

        # DataFrame oluştur
        df = pd.DataFrame(data)

        # Tabloyu görselleştir
        plt.figure(figsize=(10, 6))
        table = plt.table(cellText=df.values, colLabels=df.columns,
                         loc='center', cellLoc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(12)
        table.scale(1, 1.5)
        plt.axis('off')
        plt.title('Eğitim Özet Tablosu', fontweight='bold', y=0.9)
        plt.tight_layout()
        plt.savefig(os.path.join(cikti_dizini, 'egitim_ozeti.png'), dpi=300, bbox_inches='tight')
        plt.close('all')

## Sonuçları oku ve görselleştir

In [30]:
train_results = json_dosya_oku(train_sonuc_json_path)
eval_sonuc = json_dosya_oku(eval_sonuc_json_path)

In [31]:
# Görselleştirmeleri oluştur
print("Eğitim süreci görselleştiriliyor...")
egitim_sureci_gorsellestirilmesi(train_results)

print("Değerlendirme sonuçları görselleştiriliyor...")
degerlendirme_sonuclari_gorsellestirilmesi(eval_sonuc)

print("Eğitim özet tablosu oluşturuluyor...")
egitim_ozet_tablosu(train_results, eval_sonuc)

Eğitim süreci görselleştiriliyor...
Değerlendirme sonuçları görselleştiriliyor...
Eğitim özet tablosu oluşturuluyor...
